In [3]:
from collections import namedtuple

class P(namedtuple("Point", ["x", "y", "z"])):
    def __add__(self, other):
        return P(self.x + other.x, self.y + other.y, self.z + other.z)


In [5]:
import re
l = "-1,-1,1"
a,b,c = map(int, l.split(","))
print(a,b,c)

-1 -1 1


In [55]:
def sign(n):
    if n>0:
        return 1
    if n == 0:
        return 0
    return -1
orders = []
for d_1 in [1, -1, 2, -2, 3, -3]:
    l = [1, -1, 2, -2, 3, -3]
    l.remove(d_1)
    l.remove(-d_1)
    for d_2 in l:
        l2 = l[::]
        l2.remove(d_2)
        l2.remove(-d_2)
        d_3 = sign(d_2) * max(l2)
        orders.append((d_1, d_2, d_3))
def parse(p, order):
    return P(x=p[abs(order[0])-1]*sign(order[0]), y=p[abs(order[1])-1]*sign(order[1]), z=p[abs(order[2])-1]*sign(order[2]))


In [65]:
rot90 = {
    "x": [[1,0,0], [0,0,-1], [0,1,0]],
    "y": [[0,0,1], [0,1,0], [-1,0,0]],
    "z": [[0,-1,0], [1,0,0], [0,0,1]]
}

from math import prod
def rotate(p, ax):
    return P(*[sum(map(prod, zip(p, rot90[ax][i]))) for i in range(3)])

rotate(P(1,2,0), "y")

P(x=0, y=2, z=-1)

```x y z
z x y  // cyclic shift is two swaps
y z x

y x -z  // swap first two, change one sign
x z -y
z y -x
That's all swapping around done. Now playing with signs, in short you get 4 versions of each by adding 0/2 sign changes like

+ + +
- - +
- + -
+ - - 